In [3]:
import selenium
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
driverPATH =  r'/home/alby/Downloads/chromedriver'
browserBIN = '/usr/bin/vivaldi'

In [5]:
options = webdriver.ChromeOptions()
options.binary_location = browserBIN

In [6]:
driver = webdriver.Chrome(driverPATH, options=options)

In [91]:
url="https://www.bseindia.com/markets.html"
driver.get(url)

In [93]:
gain_table=driver.find_elements(By.XPATH,"//tbody[@class='ng-scope'and @ng-repeat='G in GainersAndLoosers.Table track by G.scrip_cd']")
links_gain=[]
links_loss=[]
for j in gain_table:

    link_tag=j.find_element(By.TAG_NAME,"a")
    links_gain.append(link_tag.get_attribute('href'))

time.sleep(5)

loss=driver.find_element(By.XPATH,'//*[@id="idLosers"]')
loss.click()

loss_table=driver.find_elements(By.XPATH,"//tbody[@class='ng-scope'and @ng-repeat='L in GainersAndLoosers1.Table track by L.scrip_cd']")


for l in loss_table:

    tag=l.find_element(By.TAG_NAME,"a")
    links_loss.append(tag.get_attribute('href'))


In [99]:
loss_table

[<selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="472b3243-8eb6-4c24-a9e0-f32a6fff9c20")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="89fd799f-a96c-4d89-bf7f-18d4a6ced113")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="b33b6780-8e64-4f41-bc71-856784d3a546")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="323679e8-3dc5-4709-bd50-11077d48244e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="aebf97ef-d81f-42e4-8c66-7a8184074f2a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="14d7ccff-588c-472d-9596-6d4dc38ea765")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b0f609b8ce2f60a248aa4f0d6c1297fb", element="1baeb682-a614-4e38-810d-36

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

query = "Davangere Sugar Company"
tweets = []
limit = 5000

In [2]:
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content])

/tmp/ipykernel_35121/1092902452.py:8: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content])


In [3]:
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
print(df)

                       Date             User  \
0 2022-06-14 09:17:08+00:00       tradingqna   
1 2022-05-18 15:20:12+00:00   MarketUpdates6   
2 2022-03-15 08:39:15+00:00         CBhoomia   
3 2022-02-16 17:05:21+00:00      equitybulls   
4 2021-11-12 10:37:02+00:00      MarketsCafe   
5 2017-06-01 12:02:40+00:00       GauriMalur   
6 2015-09-17 13:52:48+00:00  jgopikrishnan70   
7 2015-07-01 11:33:00+00:00        domexanil   
8 2012-11-27 02:21:24+00:00   Tv9kannadanews   

                                               Tweet  
0  The rights issue of Davangere Sugar Company op...  
1  Davangere Sugar Company Ltd - 543267 - Board M...  
2  👇🏾\nVT-DVG AS 350 B3 HELICOPTER 4303 06.03. 20...  
3  Davangere Sugar Company Q3FY22 profit higher a...  
4  Davangere Sugar Company reports standalone net...  
5  Local farmers say that Davangere Sugar Company...  
6  @SouleFacts This helicopter belongs to Davange...  
7  Davangere Sugar Company is planning maize star...  
8  I uploaded a @YouTube